In [28]:
import os
import re
import spacy 
from spacy import displacy
import json
import gensim
from gensim.corpora import Dictionary
from gensim.models import LdaModel, CoherenceModel, LsiModel, HdpModel
from nltk.stem import WordNetLemmatizer
from nltk.corpus import stopwords
import pandas as pd

import pyLDAvis
import pyLDAvis.gensim_models

In [55]:
stop_words = set(stopwords.words('english'))
stop_words.update(('appellant', 'respondent', 'plaintiff', 'defendant', 'mr', 'dr', 'mdm'))

In [60]:
def preprocess_text(text):
    text = str(text).lower()
    text = re.sub(r'\W', ' ', text)
    text = re.sub(r'\s+', ' ', text)
    text = re.sub(r'[0-9]', '', text)
    text = re.sub(r'\W*\b(?!no)\w{1,2}\b', '', text)
    stop_words = set(stopwords.words('english'))
    stop_words.update(('appellant', 'respondent', 'plaintiff', 'defendant', 'mr', 'dr', 'mdm'))
    lemmatizer = WordNetLemmatizer()
    words = text.split()
    words = [lemmatizer.lemmatize(word) for word in words if word not in stop_words]
    
    return words



In [61]:
filepath = 'C:/Users/gaoh7/Documents/GitHub/case-outcome-predictor/data/rawish_data/facts.json'
print("Preprocessing text")
data = []
with open(filepath, 'r') as file:
    for line in file:
        data.append(json.loads(line))
df = pd.DataFrame(data)
df = df[['casename', 'facts']]    
df['processed_facts'] = df['facts'].apply(preprocess_text)
df.drop(columns=['facts'], inplace=True)
print(df["processed_facts"])
print("Finished preprocessing text")

texts, article = [], []
print("Performing topic modelling")
for fact in df['processed_facts']:
    for word in fact:
        article.append(word)
        
    texts.append(article)
    article = []

dictionary = Dictionary(texts)
corpus = [dictionary.doc2bow(text) for text in texts]
lda_model = LdaModel(corpus=corpus, num_topics=10, id2word=dictionary)
print(lda_model.show_topics())
pyLDAvis.enable_notebook()
vis = pyLDAvis.gensim_models.prepare(lda_model, corpus, dictionary)
vis

Preprocessing text
0       [fact, widow, one, tan, geok, tee, deceased, s...
1       [fact, surrounding, circumstance, including, a...
2       [background, first, appellant, french, company...
3       [background, microsoft, adobe, autodesk, compa...
4       [fact, mere, assertion, would, suffice, exh, s...
                              ...                        
8515    [fact, first, accused, low, sze, song, low, ye...
8516    [fact, giving, opinion, representation, would,...
8517    [fact, party, karan, bagga, litigant, person, ...
8518                                                   []
8519    [fact, party, towa, company, incorporated, jap...
Name: processed_facts, Length: 8520, dtype: object
Finished preprocessing text
Performing topic modelling
[(0, '0.013*"court" + 0.009*"application" + 0.008*"order" + 0.008*"law" + 0.007*"singapore" + 0.007*"version" + 0.007*"affidavit" + 0.006*"sghc" + 0.006*"party" + 0.006*"hr"'), (1, '0.010*"would" + 0.009*"koh" + 0.008*"hook" + 0.006*"evi

PreparedData(topic_coordinates=              x         y  topics  cluster       Freq
topic                                                
8      0.087007 -0.035719       1        1  17.061790
9     -0.169931 -0.010395       2        1  12.806001
2      0.043916 -0.026782       3        1  12.641478
0      0.079186 -0.031296       4        1  12.077953
3      0.077394  0.041159       5        1  10.656623
7      0.074285 -0.009189       6        1  10.321450
6     -0.133869  0.007944       7        1   7.276148
4     -0.047833 -0.058862       8        1   6.686967
5      0.006691  0.119608       9        1   5.456550
1     -0.016846  0.003531      10        1   5.015041, topic_info=         Term          Freq         Total Category  logprob  loglift
825   accused  27411.000000  27411.000000  Default  30.0000  30.0000
3291   victim  10346.000000  10346.000000  Default  29.0000  29.0000
3805     drug   8804.000000   8804.000000  Default  28.0000  28.0000
46      share  16079.000000  16079.000000  Default  27.0000  27.0000
836    charge  14682.000000  14682.000000  Default  26.0000  26.0000
...       ...           ...           ...      ...      ...      ...
220   however    730.011838  10028.183210  Topic10  -5.8857   0.3726
715   account    766.319903  16824.339154  Topic10  -5.8371  -0.0963
595       lee    667.761000   6703.419071  Topic10  -5.9748   0.6863
52    whether    685.776286  10255.751161  Topic10  -5.9482   0.2877
9         one    688.997580  17425.312674  Topic10  -5.9435  -0.2377

[898 rows x 6 columns], token_table=       Topic      Freq        Term
term                              
15717      8  0.875780       abani
15717     10  0.083408       abani
26811      8  0.945452  absconding
5310       1  0.017404         abt
5310       3  0.017404         abt
...      ...       ...         ...
24144      7  0.945588      ziploc
2344       1  0.052026       zoned
2344       3  0.026013       zoned
2344       4  0.884441       zoned
2344       5  0.026013       zoned

[4389 rows x 3 columns], R=30, lambda_step=0.01, plot_opts={'xlab': 'PC1', 'ylab': 'PC2'}, topic_order=[9, 10, 3, 1, 4, 8, 7, 5, 6, 2])